In [7]:
from math import sqrt
import numpy as np
import matplotlib.path as mpl_path
import matplotlib.pyplot as pyplot
import pre_processing as pp

In [17]:
class GridPoint():
    def __init__(self, pid: int, x: float, y: float, score: float = 0):
        self._pid = pid
        self._x = x
        self._y = y
        self._score = score
        self._neighbours = []

    def get_coordinates(self):
        return (self._x, self._y)

    @property
    def neighbours(self):
        return self._neighbours

    @neighbours.setter
    def neighbours(self, neighbours: list):
        # print(f"set grid point ({self._x}, {self._y}) neigbours to: {neighbours}")
        self._neighbours = neighbours

    @property
    def pid(self):
        return self._pid

    @property
    def score(self):
        return self._score

    @score.setter
    def score(self, score):
        #print(f"set grid point ({round(self._x, 3)}, {round(self._y, 3)}) to score: {round(score, 3)}")
        self._score = score

    def __repr__(self):
        # return (f"{self._pid}")
        return (f"({round(self._x, 3)}, {round(self._y, 3)}, {self._score})")


def generate_point_grid(grid_resolution=20) -> list:
    VAL_RANGE = 2.88
    MIN_VAL = 1.44
    # 1. Generates a point grid G of resolution N_g = n_g x n_g within a bounding box of size ~[-1.2, 1.2].
    #    Grid size 20 x 20 deemed adequate in papag. et al. We add padding to help neighbour generation
    padded_grid = grid_resolution - 1 + 2
    point_grid = []
    point_id = 0
    for i in range(padded_grid, -1, -1):
        point_grid_row = []
        y_coord = i/padded_grid * VAL_RANGE - MIN_VAL
        for j in range(padded_grid+1):
            x_coord = j/padded_grid * VAL_RANGE - MIN_VAL
            point_grid_row.append(GridPoint(point_id, x_coord, y_coord))
            point_id += 1
        point_grid.append(point_grid_row)

    # Sets neighbours for each grid point (not including padding)
    for i in range(1, len(point_grid)-1):
        for j in range(1, len(point_grid)-1):
            neighbour_list = []
            neighbour_list.append(point_grid[i-1][j-1])     # bottom left
            neighbour_list.append(point_grid[i][j-1])       # bottom
            neighbour_list.append(point_grid[i+1][j-1])     # bottom right
            neighbour_list.append(point_grid[i+1][j])       # right
            neighbour_list.append(point_grid[i+1][j+1])     # top right
            neighbour_list.append(point_grid[i][j+1])       # top
            neighbour_list.append(point_grid[i-1][j+1])     # top left
            neighbour_list.append(point_grid[i-1][j])       # left
            point_grid[i][j].neighbours = neighbour_list

    # Flattens point grid list
    # point_grid = [item for sublist in point_grid.copy() for item in sublist]

    # Removes padding from output by removing points that were not given neighbours
    cleaned_point_grid = []
    for row in point_grid:
        cleaned_row = [x for x in row if x.neighbours]
        # Completely empty rows (first and last) are also removed
        if cleaned_row:
            cleaned_point_grid.append(cleaned_row)

    return cleaned_point_grid


def calculate_score(
        point_grid,
        internal_nodes,
        contour_points) -> None:
    # 0. Define contour (for finding points inside)
    contour_path = mpl_path.Path(contour_points)

    # For each point in point grid, iterate over internal nodes and calulcate euclidean distance.
    # If point is not in contour, set distance to 2 (penalized)
    for row in point_grid:
        for point in row:
            score = 2
            point_coordinates = point.get_coordinates()
            if contour_path.contains_point(point_coordinates):
                for internal_node in internal_nodes:
                    distance = sqrt((point_coordinates[0]-internal_node[0])
                                    ** 2 + (point_coordinates[1] - internal_node[1])**2)
                    if distance < score:
                        score = distance
            point.score = score

def generate_patches_from_contour(contour, point_grid):
    patch_size = 2
    grid_resolution = 20
    # Point grid is a flat list of a 20x20 grid filled with GridPoint-objects
    # GridPoint.score contains score of GP-object
    # GridPoint.get_coordinates() gives x- and y-coordinate tuples
    # Wanted output is list of lists, where each list contains:
    # contour coordinates [x1 -> y6] & patch coordinates [x1 -> y4] & patch scores [sg1 -> sg4]
    patches = []
    for row in range(0, grid_resolution, patch_size):
        for col in range(0, grid_resolution, patch_size):
            # p1, p2, p3, p4 = (row, col), (row, col+1), (row+1, col), (row+1, col+1) 
            patch = [] 
            p1 = point_grid[row][col]
            p2 = point_grid[row][col+1]
            p3 = point_grid[row+1][col]
            p4 = point_grid[row+1][col+1]
            
            patch_coordinates = [p1.get_coordinates(), p2.get_coordinates(),
                          p3.get_coordinates(), p4.get_coordinates()]
            
            # Flattened list of tuples
            patch.extend(contour)
            patch.extend([i for s in patch_coordinates for i in s])
            
            patch.extend([p1.score, p2.score, p3.score, p4.score])
            
            patches.append(patch)
    return patches


In [18]:
pg = generate_point_grid()
sample_points = [(0.0, 0.0)]
contour = pp.procrustes(pp.create_random_ngon(8))['transformed_contour']
calculate_score(pg, sample_points, contour)

In [19]:
import pandas as pd
import tensorflow as tf

# CONSTANTS
LEARNING_RATE = 0.001
EPOCHS = 30
BATCH_SIZE = 64

# Pre-processing
df = pd.read_csv('data/6-gon-mesh-with-internal-nodes-test.csv')
df_two_internal = df[df.internal_node_count == 2.0].dropna(axis=1, how='all')
df_two_internal = df_two_internal.drop("target_edge_length", axis=1)
df_two_internal_no_dupes = df_two_internal.drop_duplicates()
df_two_internal_no_dupes.head()

dataset = df_two_internal_no_dupes


In [20]:
# Now that we have a csv consisting of contour + specific internal node count = 2 + location of internal nodes we want to make grid score:
# for each row in csv
df_row = df_two_internal.head(1)

# Split row of dataframe into contour and internal nodes
contour_list = df_row.loc[:, 'x1':'y6'].values.tolist()[0]
internal_nodes_list = df_row.loc[:, 'a': 'd'].values.tolist()[0]
internal_node_count = 2

# Turn flat lists into list of tuples
contour = [(contour_list[i], contour_list[i+1]) for i in range(0, len(contour_list), 2)]
internal_nodes = list(zip(internal_nodes_list[::2], internal_nodes_list[1::2]))

# Point grid stuff
point_grid = generate_point_grid()
calculate_score(point_grid, internal_nodes, contour)
#contour_list, contour, internal_nodes
# point_grid


In [21]:
def patches_to_csv(contour_patches):
    with open(f"data/patches-dataset-test.csv", "w", newline="") as file:
        writer = csv.writer(file)
    pass


In [13]:
generate_patches_from_contour(contour_list, point_grid)

 0.617142857142857,
  -0.20571428571428574,
  0.754285714285714,
  -0.20571428571428574,
  0.5494679352019766,
  2,
  2,
  2],
 [0.856127350488589,
  0.2821286589889808,
  0.1594808075540417,
  0.7140957107445235,
  -0.5231279135529219,
  0.8045423321764691,
  -0.5383674713206111,
  0.0018945657113241,
  -0.4419206231140318,
  -1.4419482922874145,
  0.4878078499449342,
  -0.3607129753338832,
  0.8914285714285715,
  -0.06857142857142873,
  1.0285714285714285,
  -0.06857142857142873,
  0.8914285714285715,
  -0.20571428571428574,
  1.0285714285714285,
  -0.20571428571428574,
  2,
  2,
  2,
  2],
 [0.856127350488589,
  0.2821286589889808,
  0.1594808075540417,
  0.7140957107445235,
  -0.5231279135529219,
  0.8045423321764691,
  -0.5383674713206111,
  0.0018945657113241,
  -0.4419206231140318,
  -1.4419482922874145,
  0.4878078499449342,
  -0.3607129753338832,
  1.1657142857142855,
  -0.06857142857142873,
  1.3028571428571425,
  -0.06857142857142873,
  1.1657142857142855,
  -0.2057142857142